In [1]:
import pandas as pd
from scipy import stats
import plotly.express as px
from tqdm.notebook import tqdm

from IPython.display import display

In [2]:
raw_data_dir = '../data/raw/'
gene_exp_dir = '../data/processed/gene_expression/'
org_pairs_dir = '../data/processed/organotropism_pairs/'
intercell_net_dir = '../data/processed/intercell_networks/'
interactions_dir = '../data/processed/intercell_interactions/'
intracell_dir = '../data/processed/intracell_network/'
enrichment_dir = '../data/processed/enrichment_analysis/'

# Cancer Driver Genes (CDG) Enrichment analysis
We will use three gene lists to perform functional enrichment:
* source original: all genes found statistically significant $(\alpha=0.01)$ -> 506 genes
* source outliers: outliers from the statistically significant gene list -> 105 genes
* target outliers: outliers from the statistically significant gene list -> 1503 genes

## Load datasets

In [18]:
# load intracell genes
intracell_genes = pd.read_csv(intracell_dir+'intracell_genes.csv', names=['gene'], header=0)
intracell_genes.head(2)

,gene
0,A1BG
1,A1CF


In [19]:
# load cancer driver genes
cdg = pd.read_table(raw_data_dir+'NCG_cancerdrivers_annotation_supporting_evidence.tsv')
print('# of CDG:', cdg.symbol.drop_duplicates().shape[0])
print('# of Canonical CDG:', cdg.loc[cdg.type=='Canonical Cancer Driver', 'symbol'].drop_duplicates().shape[0])
cdg.head(2)

# of CDG: 3347
# of Canonical CDG: 591


,entrez,symbol,pubmed_id,type,organ_system,primary_site,cancer_type,method,coding_status,cgc_annotation,vogelstein_annotation,saito_annotation,NCG_oncogene,NCG_tsg
0,23,ABCF1,31444325,WGS-WES,Hematologic and lymphatic,blood,multiple_myeloma,dNdScv,coding,NaN,NaN,NaN,NaN,NaN
1,25,ABL1,29625053,Pan-cancer,Multiple,multiple,pan-cancer_adult,PanSoftWare,coding,"oncogene, fusion",Oncogene,NaN,1.0,0.0


In [20]:
# Load intercellular gene lists
intercell_target = pd.read_csv(intracell_dir+'target_labels.csv')
intercell_source = pd.read_csv(intracell_dir+'source_labels.csv')
print(intercell_target.shape[0])
display(intercell_target.head(2))
print(intercell_source.shape[0])
intercell_source.head(2)

1196


,gene,curated_label,label,is_curated
0,A1BG,0,0,True
1,ABCB1,0,0,False


1464


,gene,curated_label,label,is_curated
0,A2M,0,0,True
1,ACAN,0,0,True


In [21]:
# load intracellular gene lists
source = pd.read_csv(enrichment_dir+'source_sign.csv')
print(source.shape[0])
source_outliers = pd.read_csv(enrichment_dir+'source_sign_outliers.csv')
print(source_outliers.shape[0])
target = pd.read_csv(enrichment_dir+'target_sign.csv')
print(target.shape[0])
target_outliers = pd.read_csv(enrichment_dir+'target_sign_outliers.csv')
print(target_outliers.shape[0])

222
23
7608
1177


In [22]:
# load curated gene lists
source_curated = pd.read_csv(enrichment_dir+'source_curated_sign.csv')
print(source_curated.shape[0])
source_curated_outliers = pd.read_csv(enrichment_dir+'source_curated_sign_outliers.csv')
print(source_curated_outliers.shape[0])
target_curated = pd.read_csv(enrichment_dir+'target_curated_sign.csv')
print(target_curated.shape[0])
target_curated_outliers = pd.read_csv(enrichment_dir+'target_curated_sign_outliers.csv')
print(target_curated_outliers.shape[0])

246
23
4822
761


## Over-representation analysis with Fisher's Exact Test (Hypergeometric Test)

### Intercellular Genes

#### Complete Graph

In [27]:
gene_lists = [intercell_source, intercell_target]
labels = ['source', 'target']

cdg_enrichment = []
for label, gene_list in zip(labels, gene_lists):
    # 0: not_met genes, 1: met genes
    for i in range(2):        
        # # of genes (population size)
        N = gene_list.shape[0]
        # # of cancer driver genes present in the intercell graph (sample size)
        cdg_list = cdg.loc[cdg.symbol.isin(gene_list.gene), 'symbol'].drop_duplicates()
        n = cdg_list.shape[0]
        # # number of successes in the population (met or not_met genes)
        K = gene_list[gene_list.label==i].shape[0]
        # # number of successes in the sample (cdg & (met or not_met))
        k = gene_list[(gene_list.label==i)&(gene_list.gene.isin(cdg_list))].shape[0]
        b = K - k
        c = n - k
        d = N + k - n - K
        
        table = [[k, b], [c, d]]
        od, pval = stats.fisher_exact(table, alternative='greater')

        cdg_enrichment.append({
            'gene_type': label,
            'is_associated': i,
            'n_genes': K,
            'n_cdg': k,
            'pvalue': pval,
            'OR': od
        })

cdg_enrichment = pd.DataFrame(cdg_enrichment)
cdg_enrichment

,gene_type,is_associated,n_genes,n_cdg,pvalue,OR
0,source,0,1075,248,0.985973,0.751049
1,source,1,389,111,0.019614,1.331472
2,target,0,828,206,0.992709,0.719481
3,target,1,368,116,0.010620,1.389891


In [28]:
cdg_enrichment.to_csv(enrichment_dir+'intercell_cdg_enrichment.csv', index=False)

#### Curated Graph

In [29]:
gene_lists = [
    intercell_source[intercell_source.is_curated==True],
    intercell_target[intercell_target.is_curated==True]
]
labels = ['source', 'target']

cdg_enrichment_curated = []
for label, gene_list in zip(labels, gene_lists):
    # 0: not_met genes, 1: met genes
    for i in range(2):        
        # # of genes (population size)
        N = gene_list.shape[0]
        # # of cancer driver genes present in the intercell graph (sample size)
        cdg_list = cdg.loc[cdg.symbol.isin(gene_list.gene), 'symbol'].drop_duplicates()
        n = cdg_list.shape[0]
        # # number of successes in the population (met or not_met genes)
        K = gene_list[gene_list.curated_label==i].shape[0]
        # # number of successes in the sample (cdg & (met or not_met))
        k = gene_list[(gene_list.curated_label==i)&(gene_list.gene.isin(cdg_list))].shape[0]
        b = K - k
        c = n - k
        d = N + k - n - K
        
        table = [[k, b], [c, d]]
        od, pval = stats.fisher_exact(table, alternative='greater')

        cdg_enrichment_curated.append({
            'gene_type': label,
            'is_associated': i,
            'n_genes': K,
            'n_cdg': k,
            'pvalue': pval,
            'OR': od
        })

cdg_enrichment_curated = pd.DataFrame(cdg_enrichment_curated)
cdg_enrichment_curated

,gene_type,is_associated,n_genes,n_cdg,pvalue,OR
0,source,0,1087,253,0.981365,0.737795
1,source,1,278,81,0.026756,1.355390
2,target,0,809,205,0.986606,0.718274
3,target,1,268,86,0.019729,1.392227


In [30]:
cdg_enrichment_curated.to_csv(enrichment_dir+'intercell_cdg_enrichment_curated.csv', index=False)

### Intracellular Genes

In [31]:
# gene lists
# all genes (population size)
intracell_list = intracell_genes.gene
N = intracell_list.shape[0]
print(N)
display(intracell_list.head(2))
print()

# cancer driver genes present in the intracell graph (sample size)
cdg_list = cdg.symbol.drop_duplicates()[lambda x: x.isin(intracell_list)]
n = cdg_list.shape[0]
print(n)
display(cdg_list.head(2))

18215


0    A1BG
1    A1CF
Name: gene, dtype: object


3251


0    ABCF1
1     ABL1
Name: symbol, dtype: object

In [32]:
gene_lists = [source.gene, source_outliers.gene, target.gene, target_outliers.gene]
labels = ['source', 'source_outliers', 'target', 'target_outliers']

cdg_enrichment = []
for label, gene_list in zip(labels, gene_lists):
    
    K = gene_list.shape[0]
    k = gene_list[gene_list.isin(cdg_list)].shape[0]
    b = K - k
    c = n - k
    d = N + k - n - K
    table = [[k, b], [c, d]]
    
    od, pval = stats.fisher_exact(table, alternative='greater')
    
    cdg_enrichment.append({
        'dataset': label,
        'n_genes': gene_list.shape[0],
        'n_cdg': k,
        'pvalue': pval,
        'OR': od
    })
    
cdg_enrichment = pd.DataFrame(cdg_enrichment)
cdg_enrichment

,dataset,n_genes,n_cdg,pvalue,OR
0,source,222,69,1.027482e-06,2.099142
1,source_outliers,23,14,5.222008e-06,7.186695
2,target,7608,1428,3.187716e-03,1.113385
3,target_outliers,1177,381,6.528796e-36,2.362863


In [33]:
gene_lists = [source_curated.gene, source_curated_outliers.gene, target_curated.gene, target_curated_outliers.gene]
labels = ['source', 'source_outliers', 'target', 'target_outliers']

cdg_enrichment_curated = []
for label, gene_list in zip(labels, gene_lists):
    
    K = gene_list.shape[0]
    k = gene_list[gene_list.isin(cdg_list)].shape[0]
    b = K - k
    c = n - k
    d = N + k - n - K
    table = [[k, b], [c, d]]
    
    od, pval = stats.fisher_exact(table, alternative='greater')
    
    cdg_enrichment_curated.append({
        'dataset': label,
        'n_genes': gene_list.shape[0],
        'n_cdg': k,
        'pvalue': pval,
        'OR': od
    })
    
cdg_enrichment_curated = pd.DataFrame(cdg_enrichment_curated)
cdg_enrichment_curated

,dataset,n_genes,n_cdg,pvalue,OR
0,source,246,68,8.817985e-05,1.774610
1,source_outliers,23,13,3.470508e-05,6.003768
2,target,4822,912,1.311620e-02,1.102319
3,target_outliers,761,259,4.523286e-28,2.493807


In [34]:
a = target_curated_outliers[target_curated_outliers.gene.isin(cdg_list)]
b = source_curated_outliers[source_curated_outliers.gene.isin(cdg_list)]
pd.merge(a, b, on=['gene', 'signal'], suffixes=['_target', '_source'])

,gene,signal,pvalue_target,statistic_target,pvalue_source,statistic_source
0,JAK1,positive,0.009330,0.003056,0.017103,0.000367
1,JAK3,positive,0.007568,0.001066,0.017103,0.000122
2,LCK,positive,0.002807,0.001048,0.017103,0.000417
3,RB1,positive,0.040317,0.000035,0.041089,0.000049
4,RELA,positive,0.002807,0.000307,0.017103,0.001768


In [35]:
cdg_enrichment.to_csv(enrichment_dir+'cdg_enrichment.csv', index=False)
cdg_enrichment_curated.to_csv(enrichment_dir+'cdg_enrichment_curated.csv', index=False)

# Cancer Hallmarks enrichment analysis
Performed with R package `clusterProfiler`

## Load datasets

In [3]:
# load intracell genes
intracell_genes = pd.read_csv(intracell_dir+'intracell_genes.csv', names=['gene'], header=0)
intracell_genes.head(2)

,gene
0,A1BG
1,A1CF


In [4]:
# load gene lists
source = pd.read_csv(enrichment_dir+'source_sign.csv')
source_outliers = pd.read_csv(enrichment_dir+'source_sign_outliers.csv')
target = pd.read_csv(enrichment_dir+'target_sign.csv')
target_outliers = pd.read_csv(enrichment_dir+'target_sign_outliers.csv')

In [5]:
hallmarks2goterms = pd.read_excel(raw_data_dir+'hallmarks_to_goterms.xlsx', usecols=[0, 1, 2])
hallmarks2goterms.head(2)

,Hallmarks,GO terms,Term name
0,Sustaining Proliferative Signaling,GO:0008283,Cell Proliferation
1,Sustaining Proliferative Signaling,GO:0007049,Cell Cycle


In [6]:
hallmarks2goterms['Hallmarks'] = hallmarks2goterms['Hallmarks'].str.strip()
hallmarks2goterms['GO terms'] = hallmarks2goterms['GO terms'].str.strip()
hallmarks2goterms['Term name'] = hallmarks2goterms['Term name'].str.strip()

## Intercell Genes

In [7]:
labels = ['source', 'source_nsign', 'target', 'target_nsign']
network = ['complete', 'curated']

In [8]:
# complete network
go_enrichment = [
    pd.read_csv(enrichment_dir+'/intercell_go/source_sign_go.csv'),
    pd.read_csv(enrichment_dir+'/intercell_go/source_nsign_go.csv'),
    pd.read_csv(enrichment_dir+'/intercell_go/target_sign_go.csv'),
    pd.read_csv(enrichment_dir+'/intercell_go/target_nsign_go.csv')
]
go_enrichment[0].head(2)

,ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count,FoldEnrichment
0,GO:0006955,immune response,175/385,436/1442,7.975928e-14,2.059385e-10,1.435667e-10,ADAM17/ANGPT1/ANXA1/APOE/APP/B2M/BAG6/CADM1/CC...,175,1.503336
1,GO:0007159,leukocyte cell-cell adhesion,83/385,160/1442,4.695650e-13,6.062084e-10,4.226085e-10,ANXA1/B2M/BMP4/BMP7/CCL19/CCL2/CCL21/CCL28/CCL...,83,1.942955


In [9]:
# curated network
go_enrichment_curated = [
    pd.read_csv(enrichment_dir+'/intercell_go/source_curated_sign_go.csv'),
    pd.read_csv(enrichment_dir+'/intercell_go/source_curated_nsign_go.csv'),
    pd.read_csv(enrichment_dir+'/intercell_go/target_curated_sign_go.csv'),
    pd.read_csv(enrichment_dir+'/intercell_go/target_curated_nsign_go.csv')
]
go_enrichment_curated[0].head(2)

,ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count,FoldEnrichment
0,GO:0007159,leukocyte cell-cell adhesion,64/274,158/1347,4.028101e-10,9.985662e-07,7.623711e-07,B2M/CCL19/CCL2/CCL21/CCL5/CD160/CD274/CD4/CD40...,64,1.991315
1,GO:0002684,positive regulation of immune system process,97/274,289/1347,1.238301e-09,1.534874e-06,1.171824e-06,ADAM17/APP/B2M/BAG6/CADM1/CCL19/CCL2/CCL20/CCL...,97,1.650027


In [10]:
hallmarks_mapping = []
mapping_stats = []
for net, df_list in zip(network, [go_enrichment, go_enrichment_curated]):
    for label, df in zip(labels, df_list):
        enrichment = df.loc[df['p.adjust']<0.05, ['ID', 'Description', 'p.adjust', 'Count', 'FoldEnrichment', 'geneID']]
        mapping = pd.merge(hallmarks2goterms, enrichment, left_on='GO terms', right_on='ID')
        mapping['dataset'] = [label for i in range(mapping.shape[0])]
        mapping['network'] = [net for i in range(mapping.shape[0])]
        hallmarks_mapping.append(mapping)
        
        if '_' in label:
            is_associated = 0
        else:
            is_associated = 1
        mapping_stats.append({
            'dataset': label.split('_')[0], 
            'network': net,
            'is_associated': is_associated,
            'p.adjust<0.05': enrichment.shape[0],
            '# of hallmarks': mapping.Hallmarks.drop_duplicates().shape[0],
        })
mapping_stats = pd.DataFrame(mapping_stats)
hallmarks_mapping = pd.concat(hallmarks_mapping)
hallmarks_mapping.head(1)

,Hallmarks,GO terms,Term name,ID,Description,p.adjust,Count,FoldEnrichment,geneID,dataset,network
0,Sustaining Proliferative Signaling,GO:0007049,Cell Cycle,GO:0007049,cell cycle,0.001155,33,1.791304,ADAM17/ANXA1/APP/BAG6/BMP2/BMP4/BMP7/BTC/CCL2/...,source,complete


In [11]:
mapping_stats.sort_values(['is_associated', 'dataset'], ascending=[False, True]).set_index(['dataset', 'network', 'is_associated'])

p.adjust<0.05  # of hallmarks
dataset network  is_associated                               
source  complete 1                        606               4
        curated  1                        397               5
target  complete 1                        657               7
        curated  1                        207               5
source  complete 0                          0               0
        curated  0                          0               0
target  complete 0                          1               0
        curated  0                          0               0

In [12]:
df = hallmarks_mapping.copy()
fig = px.scatter(
    df, 
    x="FoldEnrichment",
    y="Hallmarks",
    size="Count",
    color="p.adjust",
    facet_row='dataset',
    facet_col='network',
    category_orders={
        'dataset': sorted(df.dataset.unique()),
        'Hallmarks': sorted(df.Hallmarks.unique())
    },
    color_continuous_scale=px.colors.sequential.RdBu,
    hover_name='Term name',
    size_max=20,
    height=600
)
fig.show()

In [13]:
hallmarks_mapping.to_csv(enrichment_dir+'intercell_hallmarks_mapping.csv', index=False)
mapping_stats.to_csv(enrichment_dir+'intercell_hallmarks_mapping_stats.csv', index=False)

## Intracell Genes

In [15]:
labels = ['source', 'source_out', 'target', 'target_out']
network = ['complete', 'curated']

In [14]:
# complete network
go_enrichment = [
    pd.read_csv(enrichment_dir+'intracell_go/source_go.csv'),
    pd.read_csv(enrichment_dir+'intracell_go/source_out_go.csv'),
    pd.read_csv(enrichment_dir+'intracell_go/target_go.csv'),
    pd.read_csv(enrichment_dir+'intracell_go/target_out_go.csv')
]
go_enrichment[0].head(2)

,ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count,FoldEnrichment
0,GO:0007249,I-kappaB kinase/NF-kappaB signaling,24/220,283/16046,1.023892e-12,3.339936e-09,2.737564e-09,BCL10/BCL3/BRD4/CARD19/CARD9/CFLAR/IKBKG/MALT1...,24,6.185416
1,GO:0051090,regulation of DNA-binding transcription factor...,28/220,435/16046,9.996671e-12,1.630457e-08,1.336397e-08,BCL10/BCL3/CARD9/CFLAR/CRTC3/DHX9/G3BP2/HDAC2/...,28,4.694754


In [15]:
# curated network
go_enrichment_curated = [
    pd.read_csv(enrichment_dir+'intracell_go/source_curated_go.csv'),
    pd.read_csv(enrichment_dir+'intracell_go/source_out_curated_go.csv'),
    pd.read_csv(enrichment_dir+'intracell_go/target_curated_go.csv'),
    pd.read_csv(enrichment_dir+'intracell_go/target_out_curated_go.csv')
]
go_enrichment_curated[0].head(2)

,ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count,FoldEnrichment
0,GO:0007264,small GTPase mediated signal transduction,53/245,479/16046,1.119817e-30,3.724511e-27,3.123700e-27,ABR/ARAP1/ARHGAP12/ARHGAP15/ARHGAP24/ARHGAP27/...,53,7.246713
1,GO:0051056,regulation of small GTPase mediated signal tra...,35/245,296/16046,2.809896e-21,4.672856e-18,3.919065e-18,ABR/ARAP1/ARHGAP12/ARHGAP15/ARHGAP24/ARHGAP27/...,35,7.744208


In [16]:
hallmarks_mapping = []
mapping_stats = []
for net, df_list in zip(network, [go_enrichment, go_enrichment_curated]):
    for label, df in zip(labels, df_list):
        enrichment = df.loc[df['p.adjust']<0.05, ['ID', 'Description', 'p.adjust', 'Count', 'FoldEnrichment', 'geneID']]
        mapping = pd.merge(hallmarks2goterms, enrichment, left_on='GO terms', right_on='ID')
        
        if '_' in label:
            is_outlier = 1
        else:
            is_outlier = 0
            
        mapping['dataset'] = [label.split('_')[0] for i in range(mapping.shape[0])]
        mapping['network'] = [net for i in range(mapping.shape[0])]
        mapping['is_outlier'] = [is_outlier for i in range(mapping.shape[0])]
        hallmarks_mapping.append(mapping)
        
        mapping_stats.append({
            'dataset': label.split('_')[0], 
            'network': net,
            'is_outlier': is_outlier,
            'p.adjust<0.05': enrichment.shape[0],
            '# of hallmarks': mapping.Hallmarks.drop_duplicates().shape[0],
        })
mapping_stats = pd.DataFrame(mapping_stats)
hallmarks_mapping = pd.concat(hallmarks_mapping)
hallmarks_mapping.head(1)

,Hallmarks,GO terms,Term name,ID,Description,p.adjust,Count,FoldEnrichment,geneID,dataset,network,is_outlier
0,Sustaining Proliferative Signaling,GO:0045787,Positive regulation of cell cycle,GO:0045787,positive regulation of cell cycle,0.005202,14,3.241616,BRD4/CDC42/CDC73/CHEK1/HNRNPU/HSPA2/NPM1/PLSCR...,source,complete,0.0


In [17]:
mapping_stats.sort_values(['dataset', 'is_outlier'], ascending=[False, False]).set_index(['dataset', 'network', 'is_outlier'])

p.adjust<0.05  # of hallmarks
dataset network  is_outlier                               
target  complete 1                    1878               8
        curated  1                    1745               9
        complete 0                      23               0
        curated  0                     184               3
source  complete 1                     188               3
        curated  1                     225               4
        complete 0                     258               3
        curated  0                     174               1

In [18]:
df = hallmarks_mapping[
    hallmarks_mapping.is_outlier==1
].sort_values('Hallmarks', ascending=False)
fig = px.scatter(
    df, 
    x="FoldEnrichment",
    y="Hallmarks",
    size="Count",
    color="p.adjust",
    facet_row='dataset',
    facet_col='network',
    color_continuous_scale=px.colors.sequential.RdBu,
    hover_name='Term name',
    size_max=20,
    height=600
)
fig.show()

In [19]:
hallmarks_mapping.to_csv(enrichment_dir+'intracell_hallmarks_mapping.csv', index=False)
mapping_stats.to_csv(enrichment_dir+'intracell_hallmarks_mapping_stats.csv', index=False)